In [37]:
import pandas as pd

## LLM Judge Project Level

In [ ]:
# Generated explantions and readme explanation for each Project 
df = pd.read_csv('Project-level.csv')

In [42]:
from openai import OpenAI
import pandas as pd
import json
import time 
from tqdm import tqdm  

In [ ]:
client = OpenAI(api_key = 'your openai api key')

In [ ]:
# 3. Define a function that builds the prompt for the judge LLM.
def build_judge_prompt(generated, readme):
    prompt = f"""

[THE START OF FIRST DESCRIPTION]
{readme}
[THE END OF FIRST DESCRIPTION]

[THE START OF SECOND DESCRIPTION]
{generated}
[THE END OF SECOND DESCRIPTION]

"""
    return prompt


In [103]:
SYS_PROMPT = """
Check if the first description (inside [THE START OF FIRST DESCRIPTION) or the second description (inside [THE START OF SECOND DESCRIPTION])  better explains a project? Answer should be FIRST or SECOND?
Check if they are conceptually similar? Answer should be YES or NO?

"""

In [ ]:

output_list = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    generated = row["Generated summary"]
    # accepted_answer = row["Accepted Answer Body"]
    readme = row["ground truth"]
    # 
    # Build the prompt for your judge LLM (gpt-4o-mini)
    prompt_text = build_judge_prompt(
        generated, readme
    )
    time.sleep(0.5)
    try:
        
        response = client.chat.completions.create(

        # response = openai.ChatCompletion.create(
            model="gpt-4o",  # Replace with your actual model name
            messages=[
                {"role": "system", "content": SYS_PROMPT},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0.5  # Lower temperature for consistent evaluations
        )
        
        # # Extract the raw text the LLM returned
        # for chunk in response:
        #     print(chunk.choices[0].delta.content or "", end="")
        
        evaluation_text = response.choices[0].message.content
        print(evaluation_text)
        output_list.append(output_list)
    except Exception as e:
        # Handle API/network exceptions, rate limits, etc.
        print(f"OpenAI API error on row {idx}: {e}")
        # evaluations.append("Error")
        # scores_list.append([None, None, None, None])


## LLM Judge File Level

In [ ]:
file_df = pd.read_csv('file_level_results.csv')

168

## Remove Long Files

In [333]:
file_df=file_df[file_df['baseline']!='Error Long']
file_df

,Unnamed: 0,repo,file path,code,data division,paragraphs,call relations,LOC,summary,Linked Programs,Called Programs,File Calls,baseline,granite 34b,refined summary2,business,functionality,completeness
1,7,IBM_example-health-apis,X-COBOL/X-COBOL/COBOL_Files/IBM_example-health...,IDENTIFICATION DIVISION.\n PROGRA...,WORKING-STORAGE SECTION.\n 01 WS...,"['MAINLINE', 'MAINLINE-END', 'MAINLINE-EXIT', ...",Paragraph MAINLINE calls paragraph GET-HEARTRA...,155,Inquire Visit: Select visit details from DB2 t...,NaN,NaN,NaN,### Response:\nThis is a COBOL program that re...,"1. The code is written in COBOL, a programming...",This COBOL file is designed to retrieve and pr...,0.00,1.0,1
2,8,IBM_example-health-apis,X-COBOL/X-COBOL/COBOL_Files/IBM_example-health...,IDENTIFICATION DIVISION.\n PROGRA...,WORKING-STORAGE SECTION.\n 01 WS...,"['MAINLINE', 'MAINLINE-EXIT', 'Obtain-Patient-...",Paragraph MAINLINE calls paragraph WRITE-ERROR...,144,"ADD Medication Details: To add patient's name,...",NaN,NaN,NaN,### Response:\nThis is a COBOL program that pr...,1. The code snippet is written in COBOL and is...,This COBOL program manages patient medication ...,0.25,1.0,1
3,9,IBM_example-health-apis,X-COBOL/X-COBOL/COBOL_Files/IBM_example-health...,IDENTIFICATION DIVISION.\n PROGRA...,WORKING-STORAGE SECTION.\n 01 WS...,"['MAINLINE', 'MAINLINE-END', 'MAINLINE-EXIT', ...",Paragraph MAINLINE calls paragraph WRITE-ERROR...,164,ADD Medication: Business logic for adding a n...,"['HCMADB02', 'HCMADB01']",NaN,NaN,### Response:\nThe provided COBOL code is a pr...,1. The code initializes the variables WS-HEADE...,This COBOL program manages medication and pres...,NaN,NaN,NaN
4,10,IBM_example-health-apis,X-COBOL/X-COBOL/COBOL_Files/IBM_example-health...,IDENTIFICATION DIVISION.\n PROGRA...,WORKING-STORAGE SECTION.\n 01 WS...,"['MAINLINE', 'MAINLINE-END', 'MAINLINE-EXIT', ...",Paragraph MAINLINE calls paragraph WRITE-ERROR...,154,Inquire Threshold: Business Logic to obtain pa...,['HCITDB01'],NaN,NaN,### Response:\nThis is a COBOL program that pr...,"1. This code is written in COBOL, which is a p...",This COBOL program processes patient data by i...,NaN,NaN,NaN
6,12,IBM_example-health-apis,X-COBOL/X-COBOL/COBOL_Files/IBM_example-health...,IDENTIFICATION DIVISION.\n PROGRA...,WORKING-STORAGE SECTION.\n 01 WS...,"['MAINLINE', 'MAINLINE-EXIT', 'INSERT-PATIENT-...",Paragraph MAINLINE calls paragraph WRITE-ERROR...,154,ADD Threshold: Business logic for adding a new...,['HCATDB01'],NaN,NaN,### Response:\nThe provided COBOL code is a pr...,1. The code inside the <CODE> and </CODE> tags...,"The COBOL program ""MAINLINE"" initializes varia...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,178,brazilofmux_gnucobol,X-COBOL/X-COBOL/COBOL_Files/brazilofmux_gnucob...,identification division.\n functi...,working-storage section.\n 01 mo...,['0100-main'],NaN,26,Returns the linear date for the last day of th...,NaN,NaN,NaN,### Response:\nThe provided COBOL code is a pr...,The COBOL file is used to calculate the linear...,This COBOL code is used to calculate the linea...,NaN,NaN,NaN
164,179,brazilofmux_gnucobol,X-COBOL/X-COBOL/COBOL_Files/brazilofmux_gnucob...,identification division.\n functi...,working-storage section.\n 01 ld...,['0100-main'],NaN,28,Returns the day of week for the given fixed date.,NaN,NaN,NaN,### Response:\nThis COBOL program calculates t...,The COBOL file performs some arithmetic operat...,The COBOL code is performing a series of calcu...,NaN,NaN,NaN
165,180,brazilofmux_gnucobol,X-COBOL/X-COBOL/COBOL_Files/brazilofmux_gnucob...,identification division.\n functi...,working-storage section.\n 01 ld...,['0100-main'],NaN,29,Returns a requested day of the week where the ...,NaN,NaN,NaN,### Response:\nThis is a COBOL program that ca...,### Response:\nThe COBOL file calculates the l...,The COBOL code is performing a calculation to ...,NaN,NaN,NaN
166,181,brazilofmux_gnucobol,X-COBOL/X-COBOL/COBOL_Files/brazilofmux_gnucob..

In [334]:
code_list = file_df['code']
baseline_summary  = file_df['baseline']
generated_summary = file_df['refined summary2']

In [351]:
SYS_PROMPT_FILE = """
We would like to request your feedback on the performance of two AI assistants in explaining a COBOL file.

Please rate the explanations using the following three metrics:
Purpose: The explanation correctly conveys the intent behind the file's creation and its potential usage, aligning with the information in the [FILE AUTHOR'S COMMENT].
Functionality: The description of the code functionality is correct in the [FILE AUTHOR'S COMMENT].
Clarity: The explanation is well-structured, easy to follow, and avoids ambiguity.
Each assistant receives a score on a scale of 0 to 10, where a higher score indicates better performance.

Please output two lines:

The first line contains the three values for Assistant 1’s scores, separated by commas.
The second line contains the three values for Assistant 2’s scores, separated by commas.
"""

In [352]:
# 3. Define a function that builds the prompt for the judge LLM.
def build_file_level_judge_prompt(generated, baseline,gt):
    prompt = f"""
[FILE AUTHOR'S COMMENT]
{gt}
[FILE AUTHOR'S COMMENT]  
    
[THE START OF FIRST EXPLANATION]
{baseline}
[THE END OF FIRST EXPLANATION]

[THE START OF SECOND EXPLANATION]
{generated}
[THE END OF SECOND EXPLANATION]

"""
    return prompt

In [ ]:
file_level_list = []

for idx, row in tqdm(file_df.iterrows(), total=len(file_df)):
    generated = row["refined summary2"]
    # accepted_answer = row["Accepted Answer Body"]
    baseline = row["baseline"]
    code = row['code']
    gt = row['summary']
    # Build the prompt for your judge LLM (gpt-4o-mini)
    prompt_text = build_file_level_judge_prompt(
        generated, baseline,gt
    )
    time.sleep(0.5)
    try:
        
        response = client.chat.completions.create(

        # response = openai.ChatCompletion.create(
            model="gpt-4o",  # Replace with your actual model name
            messages=[
                {"role": "system", "content": SYS_PROMPT_FILE},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0.5  # Lower temperature for consistent evaluations
        )
        
        # # Extract the raw text the LLM returned
        # for chunk in response:
        #     print(chunk.choices[0].delta.content or "", end="")
        
        evaluation_text = response.choices[0].message.content
        print(evaluation_text)
        file_level_list.append(evaluation_text)
    except Exception as e:
        # Handle API/network exceptions, rate limits, etc.
        print(f"OpenAI API error on row {idx}: {e}")
        file_level_list.append("Error")
        # scores_list.append([None, None, None, None])

In [248]:
file_level_df = pd.DataFrame(data=file_level_list,columns=['results'])

In [249]:
file_level_df['first_half'] = file_level_df['results'].apply(lambda x: x.split('\n')[0])
file_level_df['second_half']= file_level_df['results'].apply(lambda x: x.split('\n')[1])
file_level_df['baseline business purpose'] = file_level_df['first_half'].apply(lambda x: float(x.split(',')[0]))
file_level_df['generated business purpose'] = file_level_df['second_half'].apply(lambda x: float(x.split(',')[0]))
file_level_df['baseline functionality'] = file_level_df['first_half'].apply(lambda x: float(x.split(',')[1]))
file_level_df['generated functionality'] = file_level_df['second_half'].apply(lambda x: float(x.split(',')[1]))
file_level_df['baseline completeness'] = file_level_df['first_half'].apply(lambda x: float(x.split(',')[2]))
file_level_df['generated completeness'] = file_level_df['second_half'].apply(lambda x: float(x.split(',')[2]))
file_level_df = file_level_df[['results', 'baseline business purpose',
       'generated business purpose', 'baseline functionality',
       'generated functionality', 'baseline completeness',
       'generated completeness']]

In [250]:
file_level_df.to_csv('file_LLM_judge_results.csv')

In [355]:
line_lists = list(zip(*(x.split('\n') for x in file_level_list)))
baseline_list, generate_list = map(list, line_lists)

In [357]:
import re
# Function to clean the list
def clean_list(data):
    cleaned_data = []
    for item in data:
        # Remove content before ":" if it exists
        cleaned_item = re.sub(r'.*?:\s*', '', item).strip()
        cleaned_data.append(cleaned_item)
    return cleaned_data

# Clean the list
cleaned_baseline_list = clean_list(baseline_list)

In [358]:
cleaned_generate_list = clean_list(generate_list)

In [359]:
bpb, fb, cb = map(list, (list(zip(*(x.split(',') for x in cleaned_baseline_list)))))
bpb = list(map(lambda x: float(x), bpb))
fb = list(map(lambda x: float(x), fb))
cb = list(map(lambda x: float(x),cb))

In [365]:
import numpy as np
print('baseline business purpose', np.mean(bpb), np.median(bpb))
print('baseline functionality', np.mean(fb), np.median(fb))
print('baseline clarity', np.mean(cb), np.median(cb))

baseline business purpose 6.412162162162162 7.0
baseline functionality 5.6891891891891895 6.0
baseline clarity 5.797297297297297 6.0


In [366]:
bpg, fg, cg = map(list, (list(zip(*(x.split(',') for x in cleaned_generate_list)))))
bpg = list(map(lambda x: float(x), bpg))
fg = list(map(lambda x: float(x), fg))
cg = list(map(lambda x: float(x),cg))

print('generated business purpose', np.mean(bpg), np.median(bpg))
print('generated functionality', np.mean(fg), np.median(fg))
print('generated clarity', np.mean(cg), np.median(cg))

generated business purpose 6.675675675675675 7.0
generated functionality 6.304054054054054 7.0
generated clarity 6.641891891891892 7.0


In [362]:
import numpy as np
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta

# Perform Mann-Whitney U test
stat, p_value = mannwhitneyu(bpb, bpg, alternative='two-sided')

# Output the results
print("Mann-Whitney U statistic:", stat)
print("P-value:", p_value)

delta, interpretation = cliffs_delta(bpb, bpg)

# Output results
print("Cliff's Delta:", delta)
print("Effect size interpretation:", interpretation)

Mann-Whitney U statistic: 8866.0
P-value: 0.003497441511268148
Cliff's Delta: -0.19046749452154857
Effect size interpretation: small


In [363]:
# Perform Mann-Whitney U test
stat, p_value = mannwhitneyu(fb, fg, alternative='two-sided')

# Output the results
print("Mann-Whitney U statistic:", stat)
print("P-value:", p_value)

delta, interpretation = cliffs_delta(fb, fg)

# Output results
print("Cliff's Delta:", delta)
print("Effect size interpretation:", interpretation)

Mann-Whitney U statistic: 8319.0
P-value: 0.000270018730411913
Cliff's Delta: -0.2404127100073046
Effect size interpretation: small


In [364]:
# Perform Mann-Whitney U test
stat, p_value = mannwhitneyu(cb, cg, alternative='two-sided')

# Output the results
print("Mann-Whitney U statistic:", stat)
print("P-value:", p_value)

delta, interpretation = cliffs_delta(cb, cg)

# Output results
print("Cliff's Delta:", delta)
print("Effect size interpretation:", interpretation)

Mann-Whitney U statistic: 7095.5
P-value: 8.107197432129857e-08
Cliff's Delta: -0.352127465303141
Effect size interpretation: medium


## LLM Judge Function level

In [251]:
function_df  = pd.read_csv('function_level_results.csv')
len(function_df)

243

In [304]:
# 3. Define a function that builds the prompt for the judge LLM.
def build_function_level_judge_prompt(code,generated, baseline, gt):
    prompt = f"""

[COMMENT]
{gt}
[COMMENT]
        
[CODE]
{code}
[CODE]

[THE START OF FIRST EXPLANATION]
{generated}
[THE END OF FIRST EXPLANATION]

[THE START OF SECOND EXPLANATION]
{baseline}
[THE END OF SECOND EXPLANATION]



"""
    return prompt

In [307]:
SYS_PROMPT_FUNCTION = """
We would like to request your evaluate on the performance of two AI assistants in explanations to the COBOL paragraph in [CODE] and you can also refer the code inline comment in [COMMENT] to assess the correctness

Each assistant receives a score on a scale of 0 to 10, where the higher score indicates better overall performance. 
There are several important criteria when you evaluating. 
1. Specific COBOL code terms or function names should not to be directly appear in the explanations. If appear or not explain the specific cobol term, you shoud deduct the overall score.
2. If the explanation is not concise, you shoud deduct the overall score.
3. They explanation should be correct. Otherwise, deduct the overall score
Please first output two lines. 
First line only contains the an overall score for Assistant 1
Second line only contains the an overall score for Assistant 2
"""

In [ ]:
function_level_list = []

for idx, row in tqdm(function_df.iterrows(), total=len(function_df)):
    generated = row["refined summary2"]
    # accepted_answer = row["Accepted Answer Body"]
    baseline = row["baseline"]
    code = row['code']
    GT = row['summary']
    # Build the prompt for your judge LLM (gpt-4o-mini)
    prompt_text = build_function_level_judge_prompt(
        code, generated, baseline,GT
    )
    time.sleep(0.5)
    try:
        
        response = client.chat.completions.create(

        # response = openai.ChatCompletion.create(
            model="gpt-4o",  # Replace with your actual model name
            messages=[
                {"role": "system", "content": SYS_PROMPT_FUNCTION},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0.5  # Lower temperature for consistent evaluations
        )
        
        # # Extract the raw text the LLM returned
        # for chunk in response:
        #     print(chunk.choices[0].delta.content or "", end="")
        
        evaluation_text = response.choices[0].message.content
        print(evaluation_text)
        function_level_list.append(evaluation_text)
    except Exception as e:
        # Handle API/network exceptions, rate limits, etc.
        print(f"OpenAI API error on row {idx}: {e}")
        function_level_list.append("Error")

In [313]:
line_lists_function = list(zip(*(x.split('\n') for x in function_level_list)))
generate_list_function,baseline_list_function, = map(list, line_lists_function)

In [314]:
b = list(map(lambda x: int(x), baseline_list_function))
g = list(map(lambda x: int(x), generate_list_function))

In [316]:
import numpy as np
print('baseline', np.mean(b), np.median(b))
print('generated', np.mean(g), np.median(g))
# print('baseline External', np.mean(e_Fb), np.median(e_Fb))
# print('baseline Constriants', np.mean(con_Fb), np.median(con_Fb))

baseline 5.567901234567901 5.0
generated 6.012345679012346 6.0


In [317]:
# f_Fg, c_Fg= map(list, (list(zip(*(x.split(',') for x in generate_list_function)))))
# f_Fg = list(map(lambda x: float(x), f_Fg))
# c_Fg = list(map(lambda x: float(x), c_Fg))
# # e_Fg = list(map(lambda x: float(x),e_Fg))
# # con_Fg = list(map(lambda x: float(x),con_Fg))
# print('genreated Functionality', np.mean(f_Fg), np.median(f_Fg))
# print('genreated Completeness', np.mean(c_Fg), np.median(c_Fg))
# # print('baseline External', np.mean(e_Fg), np.median(e_Fg))
# # print('baseline Constriants', np.mean(con_Fg), np.median(con_Fg))

In [318]:
import numpy as np
from scipy.stats import mannwhitneyu
from cliffs_delta import cliffs_delta

# Perform Mann-Whitney U test
stat, p_value = mannwhitneyu(b, g, alternative='two-sided')

# Output the results
print("Mann-Whitney U statistic:", stat)
print("P-value:", p_value)

delta, interpretation = cliffs_delta(b, g)

# Output results
print("Cliff's Delta:", delta)
print("Effect size interpretation:", interpretation)

Mann-Whitney U statistic: 24634.0
P-value: 0.0011256128316802898
Cliff's Delta: -0.16564209385425663
Effect size interpretation: small


In [321]:
import csv

with open("function_LLM_judge_results_v2.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(function_level_list)